In [1]:
# main URL area, contain for each novel 
url = "https://www.xbiquge.so/book/49549/"

In [2]:
# import area
import requests
from bs4 import BeautifulSoup
import numpy as np
import threading

/Users/tianqiliu/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# Parse the html text and retreive all the chapters 
r = requests.get(url)
soup = BeautifulSoup(r.text)

# Since BiQuGe use cache in their website, we need to drop first 12 website
# which are the newest 12 chapters and have been included later
sub_url_list = soup.find(id='list').find_all('a')[12:]
sub_url_list = [sub_url.get("href") for sub_url in sub_url_list]
title = str(soup.find(id='list').find("dt")).split("》")[0].split("《")[1]
lock = threading.Lock()
chapter_lab = {}
try:
    f = open(f"{title}_cache.txt", "r")
    cache = f.read()
    sub_url_list = sub_url_list[sub_url_list.index(cache)+1:]
    if not sub_url_list:
        print("No updated chapters")
except:
    sub_url_list = sub_url_list

In [4]:
# By checking the corresponding website to retrieve content
def get_text_and_title(url, sub_url):
#     sub_url = sub_url_list[0]
    chapter_url = url + sub_url
    r = requests.get(chapter_url)
    soup = BeautifulSoup(r.text)
    sub_text = " \n".join(soup.find(id='content').text.split("\xa0\xa0\xa0\xa0")[1:])
    sub_title = soup.find("title").text.split("_")[0]
    return sub_text, sub_title

In [5]:
# the function used in multi-thread
def get_list_text(selected_list):
    global chapter_lab, lock
    for index, sub_url in selected_list:
        sub_text, sub_title = get_text_and_title(url, sub_url)
        lock.acquire()
        chapter_lab[int(index)] = [sub_title, sub_text]
        lock.release()
    return

In [6]:
workers = 100
# sub_url_list = sub_url_list[:100]
indexed_list = [[index, i] for index, i in enumerate(sub_url_list)]
n_worker_pools = [[] for _ in range(workers)]
# Let n_workers to work as a pipeline, so that we do not need to wait one worker to finish
# his parts and then the next
for index, link in enumerate(sub_url_list):
    n_worker_pools[index%workers].append([index, link])
# First edition, we need to wait the first worker done and then move to the next
# n_worker_pools = list(np.array_split(indexed_list, workers))
for pool in n_worker_pools:
    threading.Thread(target=get_list_text, args=(pool,)).start()

In [12]:
index = 0
f = open(f"{title}.txt", "w")
# while index < len(indexed_list):
while index < 100:
    lock.acquire()
    if index in chapter_lab:
        sub_title, sub_text = chapter_lab[index]
        f.write("\n ------- \n")
        f.write("\n\n # "+sub_title + " \n\n")
        print(f"Writing chapter {sub_title}")
        f.write(sub_text)
        f.write("\n")
        index += 1
    lock.release()
f.close()
print("Done")

Writing chapter 第一章 我只想当个窝囊废
Writing chapter 第二章 颠倒黑白
Writing chapter 第三章 好
Writing chapter 第四章 是因为她
Writing chapter 第五章 对不起
Writing chapter 第六章 签订合同
Writing chapter 第七章 愿赌服输
Writing chapter 第八章 负责人变更
Writing chapter 第九章 鸡贼的人
Writing chapter 第十章 强势
Writing chapter 第十一章 提线木偶
Writing chapter 第十二章 士别三日
Writing chapter 第十三章 神秘买家
Writing chapter 第十四章 狗眼看人低
Writing chapter 第十五章 真付钱了！
Writing chapter 第十六章 意外走红
Writing chapter 第十七章 再来一辆
Writing chapter 第十八章 包下餐厅
Writing chapter 第十九章 哪个王八蛋
Writing chapter 第二十章 二十二号到了
Writing chapter 第二十一章 玫瑰飘零，轰动云城
Writing chapter 第二十二章 酣睡的猛虎
Writing chapter 第二十三章 麻烦降临
Writing chapter 第二十四章 快了，快乐
Writing chapter 第二十五章 阴险计划
Writing chapter 第二十六章 死字怎么写
Writing chapter 第二十七章 真的是窝囊废吗？
Writing chapter 第二十八章 污蔑
Writing chapter 第二十九章 偏袒
Writing chapter 第三十章 墨阳
Writing chapter 第三十一章 一人足矣
Writing chapter 第三十二章 云城，我回来了
Writing chapter 第三十三章 奇葩的一家三口
Writing chapter 第三十四章 睁眼说瞎话
Writing chapter 第三十五章 不知死活的刘花
Writing chapter 第三十六章 找他帮忙？
Writing chapter 第三十七章 都是摔倒的
Writing ch

In [8]:
f = open(f"{title}_cache.txt", "w")
f.write(sub_url_list[-1])
f.close()

In [11]:
# Debug use
# lock.release()